In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from mpmath import besseljzero
from scipy.special import gamma
from scipy.special import jv, iv, ive
from scipy.optimize import minimize
from scipy.optimize import differential_evolution
from scipy.interpolate import interp1d

from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def simulate_HSDM_2D(a, mu, ndt, sigma=1, dt=0.001):
    x = np.zeros(mu.shape)
    
    rt = 0
    
    while np.linalg.norm(x, 2) < a(rt):
        x += mu*dt + sigma*np.sqrt(dt)*np.random.normal(0, 1, mu.shape)
        rt += dt
    
    theta = np.arctan2(x[1], x[0])   
    
    return ndt+rt, theta

In [3]:
def k(a, da, t, q, sigma=2):
    return 0.5 * (q - 0.5*sigma - da(t))

def psi(a, da, t, z, tau, q, sigma=2):
    kk = k(a, da, t, q, sigma)
    
    if 2*np.sqrt(a(t)*z)/(sigma*(t-tau))<=700:
        term1 = 1./(sigma*(t - tau)) * np.exp(- (a(t) + z)/(sigma*(t-tau)))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = iv(q/sigma-1, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * iv(q/sigma, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
    else:
        term1 = 1./(sigma*(t - tau))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = ive(q/sigma-1, (a(t) + z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * ive(q/sigma, (a(t) + z)/(sigma*(t-tau)))
    
    return term1 * term2 * (term3 * term4 + term5)

def ie_bessel_fpt(a, da, q, z, sigma=2, dt=0.1, T_max=2):
    g = [0]
    T = [0]
    g.append(-2*psi(a, da, dt, z, 0, q, sigma))
    T.append(dt)
    
    for n in range(2, int(T_max/dt)+2):
        s = -2 * psi(a, da, n*dt, z, 0, q, sigma)

        for j in range(1, n):
            if a(j*dt) == 0:
                continue
            
            s += 2 * dt * g[j] * psi(a, da, n*dt, a(j*dt), j*dt, q, sigma)

        g.append(s)
        T.append(n*dt)
        
    g = np.asarray(g)
    T = np.asarray(T)
    
    gt = interp1d(T, g)
    return gt

In [4]:
def HSDM_2D_likelihood(prms, RT, Theta):
    a = lambda t: prms[0] - prms[1]*t
    a2 = lambda t: (a(t))**2
    da2 = lambda t: -2*prms[1] * a(t)
    
    ndt = prms[2]
    mu = np.array([prms[3], prms[4]])
        
    T_max = min(max(RT), prms[0]/prms[1])
    fpt = ie_bessel_fpt(a2, da2, mu.shape[0], 0.000001, 
                        dt=0.02, T_max=T_max)
        
    log_lik = 0
    for i in range(len(RT)):
        rt, theta = RT[i], Theta[i]
        if rt - ndt > 0.0001 and rt - ndt < T_max:
            mu_dot_x0 = mu[0]*np.cos(theta)
            mu_dot_x1 = mu[1]*np.sin(theta)
            term1 = a(rt - ndt) * (mu_dot_x0 + mu_dot_x1)
            term2 = 0.5 * np.linalg.norm(mu, 2)**2 * (rt - ndt)

            density = np.exp(term1 - term2) * fpt(rt - ndt)
            
            if 0.1**14 < density:
                log_lik += -np.log(density)
            else:
                log_lik += -np.log(0.1**14)
        else:
            log_lik += -np.log(0.1**14)
        
    return log_lik

In [5]:
recovery_df = {'b0_true': [],
               'b0_estimate': [],
               'lambda_true': [],
               'lambda_estimate': [],
               'ndt_true': [],
               'ndt_estimate': [],
               'mu1_true': [],
               'mu1_estimate': [],
               'mu2_true': [],
               'mu2_estimate': []}

min_b0 = 2
max_b0 = 5

min_lambda = .1
max_lambda = 2

min_ndt = 0.1
max_ndt = 1

min_mu = -3
max_mu = 3

In [6]:
for n in tqdm(range(15)):
    b0 = np.random.uniform(min_b0, max_b0)
    lamb = np.random.uniform(min_lambda, max_lambda)
    a = lambda t: b0 - lamb*t
    
    ndt = np.random.uniform(min_ndt, max_ndt)
    mu = np.array([np.random.uniform(min_mu, max_mu), 
                   np.random.uniform(min_mu, max_mu)])
    
    RT = []
    Theta = []
    
    for i in range(500):
        rt, theta = simulate_HSDM_2D(a, mu, ndt)
        RT.append(rt)
        Theta.append(theta)
    
    min_ans = differential_evolution(HSDM_2D_likelihood,
                                     args=(RT, Theta),
                                     bounds=[(min_b0, max_b0), (min_lambda, max_lambda), 
                                             (min_ndt, max_ndt), (min_mu, max_mu), (min_mu, max_mu)])
    
    min_ans = minimize(HSDM_2D_likelihood,
                       args=(RT, Theta),
                       method='nelder-mead',
                       x0=min_ans.x,
                       bounds=[(min_b0, max_b0), (min_lambda, max_lambda),
                               (min_ndt, max_ndt), (min_mu, max_mu), (min_mu, max_mu)])
    
    if min_ans.success:
        recovery_df['b0_true'].append(b0)
        recovery_df['lambda_true'].append(lamb)
        recovery_df['ndt_true'].append(ndt)
        recovery_df['mu1_true'].append(mu[0])
        recovery_df['mu2_true'].append(mu[1])

        recovery_df['b0_estimate'].append(min_ans.x[0])
        recovery_df['lambda_estimate'].append(min_ans.x[1])
        recovery_df['ndt_estimate'].append(min_ans.x[2])
        recovery_df['mu1_estimate'].append(min_ans.x[3])
        recovery_df['mu2_estimate'].append(min_ans.x[4])

100%|██████████████████████████████| 15/15 [1:16:16<00:00, 305.10s/it]


In [7]:
recovery_df = pd.DataFrame(recovery_df)
recovery_df.corr()

,b0_true,b0_estimate,lambda_true,lambda_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate
b0_true,1.000000,0.897702,0.055573,-0.185273,-0.071721,-0.015265,0.071766,0.064345,0.417024,0.396572
b0_estimate,0.897702,1.000000,-0.073318,-0.146687,-0.082426,-0.122483,0.013584,0.011916,0.446276,0.430078
lambda_true,0.055573,-0.073318,1.000000,0.892856,-0.175434,-0.104499,-0.272289,-0.268993,0.240081,0.227386
lambda_estimate,-0.185273,-0.146687,0.892856,1.000000,-0.077898,-0.083799,-0.276833,-0.262634,0.121503,0.117222
ndt_true,-0.071721,-0.082426,-0.175434,-0.077898,1.000000,0.974351,-0.022861,-0.027282,-0.326379,-0.300759
ndt_estimate,-0.015265,-0.122483,-0.104499,-0.083799,0.974351,1.000000,0.000984,-0.005922,-0.337876,-0.314786
mu1_true,0.071766,0.013584,-0.272289,-0.276833,-0.022861,0.000984,1.000000,0.998979,-0.269587,-0.269762
mu1_estimate,0.064345,0.011916,-0.268993,-0.262634,-0.027282,-0.005922,0.998979,1.000000,-0.274898,-0.275334
mu2_true,0.417024,0.446276,0.240081,0.121503,-0.326379,-0.337876,-0.269587,-0.274898,1.000000,0.998547
mu2_estimate,0.396572,0.430078,0.227386,0.117222,-0.300759,-0.314786,-0.269762,-0.275334,0.998547,1.000000


In [8]:
file_name = '_Recovery_data/IE_2d_recovery_linear_500_02.csv'
old_recovery_data = pd.read_csv(file_name, index_col=0)
recovery_df = pd.concat([old_recovery_data, 
                         recovery_df]).reset_index(drop=True)
recovery_df.to_csv(file_name)

In [9]:
recovery_df

,b0_true,b0_estimate,lambda_true,lambda_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate
0,4.397535,4.185360,0.323693,0.153169,0.831266,0.894517,-1.742754,-1.807236,1.411701,1.513183
1,4.959222,4.997928,1.074559,1.094837,0.109113,0.120593,0.873473,0.878453,-0.456587,-0.436137
2,2.272336,2.204882,0.564206,0.577438,0.481430,0.529430,1.500674,1.550882,1.012239,1.035146
3,2.432943,2.699682,0.613124,0.914689,0.312189,0.280516,-2.387932,-2.296537,-1.449686,-1.348349
4,4.941434,3.825240,1.301521,0.828797,0.731124,0.915571,-2.585319,-2.696645,1.283719,1.348807
...,...,...,...,...,...,...,...,...,...,...
95,4.991357,5.000000,0.711407,0.414071,0.470570,0.481349,-2.748873,-2.998718,2.546657,2.754672
96,3.183206,3.375541,0.119265,0.100000,0.903392,0.868392,-1.076423,-1.145130,-1.178773,-1.199641
97,4.090844,4.967774,0.913875,1.248208,0.252026,0.128424,-2.776161,-2.747215,0.602708,0.550394
98,3.561660,4.180864,0.454633,0.707342,0.378989,0.270652,2.336090,2.282342,-0.625541,-0.605711
